In [ ]:
import fnmatch
import os
import numpy as np
import pandas as pd
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
np.random.seed(21)

path = '../input/train/'
train_label = []
train_img = []
label_enum = {'Loose Silky-bent':0, 'Charlock':1, 'Sugar beet':2, 'Small-flowered Cranesbill':3,
             'Common Chickweed':4, 'Common wheat':5, 'Maize':6, 'Cleavers':7, 'Scentless Mayweed':8,
             'Fat Hen':9, 'Black-grass':10, 'Shepherds Purse':11}
for i in os.listdir(path):
    label_number = label_enum[i]
    new_path = path+i+'/'
    for j in fnmatch.filter(os.listdir(new_path), '*.png'):
        temp_img = image.load_img(new_path+j, target_size=(128,128))
        train_label.append(label_number)
        temp_img = image.img_to_array(temp_img)
        train_img.append(temp_img)
train_img = np.array(train_img)

train_y=pd.get_dummies(train_label)
train_y = np.array(train_y)
train_img=preprocess_input(train_img)

print('Training data shape: ', train_img.shape)
print('Training labels shape: ', train_y.shape)

In [ ]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D

def easy_model(num_classes=None):
    model = Sequential()
    model.add(Conv2D(32, 5, input_shape=(128, 128, 3), activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, 3, activation= 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [ ]:
from keras import backend as K
num_classes = 12
model = easy_model(num_classes)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.1, random_state=42)

from keras.callbacks import ModelCheckpoint
epochs = 10
batch_size = 16

model.fit(X_train,Y_train,
          batch_size=16,
          epochs=20,
          verbose=1, shuffle=True, validation_data=(X_valid,Y_valid))

In [ ]:
prob = []
num = []
test_img = []
test_path = '../input/test/'
test_data = fnmatch.filter(os.listdir(test_path), '*.png')

test_img = []
for i in range(len(test_data)):
    path = test_path+'/'+test_all[i]
    temp_img = image.load_img(path,target_size=(128,128))
    temp_img = image.img_to_array(temp_img)
    test_img.append(temp_img) 
test_img = np.array(test_img)    
test_img = preprocess_input(test_img)


test_labels = []
pred = model.predict(test_img)
enum_labels =  {0:'Loose Silky-bent', 1:'Charlock',2: 'Sugar beet',3: 'Small-flowered Cranesbill',
              4:'Common Chickweed',5: 'Common wheat',6: 'Maize', 7:'Cleavers', 8:'Scentless Mayweed',
             9: 'Fat Hen', 10:'Black-grass', 11:'Shepherds Purse'}

for i in range(len(test_data)):
    max_score = 0
    lab = -1
    for j in range(12):
        if pred[i][j]>max_score:
            max_score = pred[i][j]
            lab = j
    test_labels.append(enum_labels[lab])


d = {'file': test_all, 'species': test_labels}
df = pd.DataFrame(data=d)
print(df.head(5))

In [ ]:
#Convert dataframe to csv
df.to_csv("submit.csv",index=False) 

#Submit the csv
print('Submitting csv')